In [12]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import h5py

In [13]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [15]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 209
number of test examples = 50
X_train shape: (209, 64, 64, 3)
Y_train shape: (209, 1)
X_test shape: (50, 64, 64, 3)
Y_test shape: (50, 1)


In [16]:

def CatModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    ### START CODE HERE ###
    # Feel free to use the suggested outline in the text above to get started, and run through the whole
    # exercise (including the later portions of this notebook) once. The come back also try out other
    # network architectures as well. 
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    ### END CODE HERE ###
    
    return model

In [17]:
catModel = CatModel(X_train.shape[1:])

In [18]:
catModel.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [19]:
catModel.fit(x = X_train, y = Y_train, epochs = 25, batch_size = 8)

Epoch 1/25
209/209 [==============================] - 2s 10ms/step - loss: 2.9256 - acc: 0.6268
Epoch 2/25
209/209 [==============================] - 2s 9ms/step - loss: 2.2599 - acc: 0.6507
Epoch 3/25
209/209 [==============================] - 2s 9ms/step - loss: 1.6087 - acc: 0.6794
Epoch 4/25
209/209 [==============================] - 2s 9ms/step - loss: 1.4770 - acc: 0.7225
Epoch 5/25
209/209 [==============================] - 2s 9ms/step - loss: 1.0002 - acc: 0.7464
Epoch 6/25
209/209 [==============================] - 2s 9ms/step - loss: 0.5064 - acc: 0.8804
Epoch 7/25
209/209 [==============================] - 2s 9ms/step - loss: 0.2922 - acc: 0.9187
Epoch 8/25
209/209 [==============================] - 2s 9ms/step - loss: 0.3224 - acc: 0.8995
Epoch 9/25
209/209 [==============================] - 2s 9ms/step - loss: 0.2941 - acc: 0.8852
Epoch 10/25
209/209 [==============================] - 2s 9ms/step - loss: 0.3270 - acc: 0.8995
Epoch 11/25
209/209 [===========================

In [20]:
### START CODE HERE ### (1 line)
preds = catModel.evaluate(x = X_test, y = Y_test)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

50/50 [==============================] - 0s 4ms/step

Loss = 2.488178195953369
Test Accuracy = 0.54


In [21]:
img_path = 'images/cat.1.jpg'
### END CODE HERE ###
img = image.load_img(img_path, target_size=(64, 64))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(catModel.predict(x))

[[1.]]
